In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
import string
import requests
import time
import urllib3
from emoji import replace_emoji
import emoji
from bs4 import BeautifulSoup
import re
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import datetime, timedelta

def remove_emoji(text):
    # 匹配所有表情符号的正则表达式
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"  # 表情符号 (如笑脸)
        "\U0001F300-\U0001F5FF"  # 各种符号和图形 (如气象符号、交通符号等)
        "\U0001F680-\U0001F6FF"  # 交通工具和地图图标
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002700-\U000027BF"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed characters
        "\U0001F1E6-\U0001F1FF"  # Flags (iOS)
        "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', text)
def clean_text(text):
    # 移除所有中文标点符号
    text = re.sub(r'[，。！？、；：“”‘’（）【】『』《》〔〕﹏﹑︿︰﹔]', ' ', text)

    # 移除单个字母和数字
    text = re.sub(r'\b[a-zA-Z0-9]\b', ' ', text)
    text=re.sub(r'[^a-zA-Z0-9\s\n\t]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    # 去除多余空格
    text = re.sub(r'\s+', ' ', text).strip()
    # 移除单个字母和数字
    text = re.sub(r'\b[a-zA-Z0-9]\b', ' ', text)
    text=re.sub(r'[^a-zA-Z0-9\s\n\t]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    # 去除多余空格
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_content(content):
#   移除utf-8以外的字符
    content=content.encode("utf-8", errors="replace").decode("utf-8")

    content = re.sub(r'[^\x00-\x7F]', ' ', content)

    content = re.sub(r'[\t\r\v]', ' ', content)  # 移除制表符、回车符等
    
    # 清除emoji符号
    content = emoji.replace_emoji(content, replace=" ")
    content=remove_emoji(content)

    content=clean_text(content)
    

    return content

# 删除 rare_words 中的内容的函数
def remove_rare_words(text, rare_words):
    # 去除标点符号的表
    translator = str.maketrans('', '', string.punctuation)
    
    # 拆分句子并清理单词
    words = text.split()
    cleaned_words = []
    for word in words:
        # 清理当前单词（去标点符号和空格）
        clean_word = word.translate(translator).strip()
        # 如果清理后的单词不在稀有词列表中，保留它
        if clean_word not in rare_words:
            cleaned_words.append(clean_word)
    
    # 返回处理后的句子
    return ' '.join(cleaned_words)


# 初始化停用词和Snowball词干提取器
stop_words = set(stopwords.words('english'))
snowball_stemmer = SnowballStemmer("english")

# 数据预处理函数 包含词干提取 
def text_preprocessing(text):

    text=clean_content(text)
    
    # 转换为小写并词元化
    words = word_tokenize(text.lower())
    
    # 去除停用词并执行词干提取
    words = [snowball_stemmer.stem(word) for word in words if word not in stop_words]
    
    return words

In [1]:
import nltk
nltk.download('stopwords')
print(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to D:\nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:

import pandas as pd
import re
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# 1. 加载数据
data = pd.read_csv('..\\..\\Code\\RQ3 Analysis and Visualization\\Topic Model Results\\stack_overflow_(5)_type_posts.csv',encoding="ISO-8859-1")



# 对 data['Title_text'] 列预处理
data['Processed_Text'] = data['title'].apply(text_preprocessing)

rare_words=['api', 'openai', 'use','error'] 
def filter_rare_words(word_list):
    return [word for word in word_list if word not in rare_words]

# 应用函数到 'Processed_Text' 列
data['Processed_Text'] = data['Processed_Text'].apply(filter_rare_words)


# 3. 构建词典和语料库
dictionary = corpora.Dictionary(data['Processed_Text'])
dictionary.filter_extremes(no_below=2, no_above=0.9)
corpus = [dictionary.doc2bow(text) for text in data['Processed_Text']]

# 4. 选择最佳主题数 K
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus=corpus, 
                            id2word=dictionary, 
                            num_topics=num_topics, 
                            # alpha='auto', 
                            # alpha=(50/num_topics), 
                            alpha=0.01, 
                            # eta='auto', 
                            eta=0.01, 
                            iterations=1000,
                            passes=400 ,
                            random_state=42)
        model_list.append(model)
        coherence_model = models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())
    return model_list, coherence_values


optimal_k=6

# 选择最佳 K 并训练最终模型

lda_model = models.LdaModel(corpus=corpus, 
                            id2word=dictionary, 
                            num_topics=optimal_k, 
                            # alpha='auto', 
                            # alpha=(50/optimal_k), 
                            alpha=0.01, 
                            # eta='auto', 
                            eta=0.01, 
                            iterations=1000,
                            passes=120 ,
                            random_state=42)

# 5. 输出每个主题的关键词
topics = lda_model.print_topics(num_words=20)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

# 6. 为每个帖子分配主题
data['Dominant_Topic'] = [max(lda_model[doc], key=lambda x: x[1])[0] for doc in corpus]

# 保存结果
data.to_csv('..\\..\\Code\\RQ3 Analysis and Visualization\\Topic Model Results\\stack_overflow_(5)_type_posts_LDA.csv', index=False,encoding="ISO-8859-1")

Topic 0: 0.109*"whisper" + 0.067*"audio" + 0.053*"ai" + 0.049*"file" + 0.037*"transcrib" + 0.035*"correct" + 0.031*"open" + 0.031*"find" + 0.029*"stream" + 0.024*"system" + 0.024*"requir" + 0.023*"work" + 0.021*"hug" + 0.021*"creat" + 0.021*"face" + 0.017*"specifi" + 0.017*"way" + 0.017*"attributeerror" + 0.017*"numpi" + 0.017*"larg"
Topic 1: 0.080*"whisper" + 0.046*"audio" + 0.041*"file" + 0.040*"call" + 0.039*"format" + 0.037*"model" + 0.036*"send" + 0.029*"respons" + 0.028*"python" + 0.025*"ffmpeg" + 0.024*"convert" + 0.024*"translat" + 0.023*"data" + 0.022*"attribut" + 0.022*"wav" + 0.022*"modul" + 0.020*"transcript" + 0.018*"add" + 0.018*"winerror" + 0.018*"tts"
Topic 2: 0.117*"whisper" + 0.072*"text" + 0.046*"speech" + 0.038*"get" + 0.036*"instal" + 0.032*"word" + 0.028*"languag" + 0.028*"tri" + 0.028*"gpu" + 0.025*"app" + 0.025*"nativ" + 0.025*"react" + 0.025*"transcript" + 0.024*"model" + 0.021*"level" + 0.018*"fine" + 0.018*"array" + 0.018*"code" + 0.017*"file" + 0.015*"transc

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

# # 生成可视化数据
# vis_data = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

# # 显示交互式图表
# pyLDAvis.show(vis_data)


vis_data = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis_data, '..\\..\\Code\\RQ3 Analysis and Visualization\\Topic Model Results\\stack_overflow_(5)_type_posts_LDA.html')
# pyLDAvis.show(vis_data,open_browser=False,local=False)
